In [10]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind

### Getting Test Playlist From Spotify

In [2]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [3]:
print(token)

BQDIU5idfyG85qHyv17Lhf2m2sq8JJmSAPlwGPotnM-tciVjeoz1WeYgoV56ikMNyqv99YQwYty38N7UCKKi2TcdaHsyE23xrfqT-eAD1IXEYEZBaTvtIhmSjzXZMZ3rHaGVWONF2GoAhR9TJYoTzFXgNWY


In [4]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

#### Vodafone FM 

https://open.spotify.com/playlist/5QZyi299jBblzaTFW1nP4h?si=PawXR_5mSL-ZfezTCUX2Hg

In [5]:
#I'm going to test with a playlist from Vodafone FM 

playlist_index = 0

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri

#just to confirm: spotify:playlist:3M2gjOU19KrxZOCYtVYUCY

'spotify:user:Test_1:playlist:3M2gjOU19KrxZOCYtVYUCY'

In [6]:
uri = playlist_uri    # the URI is split by ':' to get the username and playlist ID
username = uri.split(':')[2]
playlist_id = uri.split(':')[4]

In [7]:
results = {'items':[]}

for n in range(0,3000,100):
    new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
    results['items'] += new['items']
    
    playlist_tracks_data = results
    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    
    for track in playlist_tracks_data['items']:
        playlist_tracks_id.append(track['track']['id'])
        playlist_tracks_titles.append(track['track']['name'])
        
    #adds a list of all artists involved in the song to the list of artists for the playlist
        for artist in track['track']['artists']:
            artist_list = []
            artist_list.append(artist['name'])
        playlist_tracks_artists.append(artist_list[0])
        
    
print(len(playlist_tracks_id))

test_df = pd.DataFrame([])
for i in range(0, len(playlist_tracks_id)):
    features = sp.audio_features(playlist_tracks_id[i])
    features_df = pd.DataFrame(features)
    test_df = test_df.append(features_df)

1190


In [8]:
#Merging Dataframes for getting audio features and track information

test_df['title'] = playlist_tracks_titles
#features_df['first_artist'] = playlist_tracks_first_artists
test_df['main_artist'] = playlist_tracks_artists
#features_df = features_df.set_index('id')
test_df = test_df[['id', 'title', 'main_artist',
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'acousticness', 'instrumentalness',
                           'liveness', 'valence', 'tempo',
                           'duration_ms', 'time_signature']]

test_df

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.625000,0.03550,0.1160,0.502,143.802,216004,4
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.044300,0.00826,0.1420,0.369,125.389,177520,4
0,6hlJIYn5IBNqcSGTS6X5Xq,My Girls,Animal Collective,0.508,0.953,4,-6.031,0,0.256000,0.04450,0.9290,0.555,119.973,340787,4
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.057700,0.01970,0.3430,0.537,117.783,315200,4
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.111000,0.04490,0.1050,0.301,139.389,335333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,30UWPcZN1mP0P6ZNy7euw1,Putos Bons,Linda Martini,0.177,0.932,0,-5.951,1,0.000007,0.69600,0.3450,0.166,185.187,289007,3
0,7zqhUgWpWzwYnT9eewPJEK,The End,Daughter,0.463,0.688,9,-8.127,1,0.873000,0.00456,0.0666,0.200,128.986,329647,4
0,6ajamBF0Lrmizka3L7dkkE,Marigold,Nice Weather for Ducks,0.304,0.681,0,-6.739,1,0.102000,0.00000,0.0853,0.625,146.049,203053,5
0,013jXbIwEakb8BWbK0bfjO,Someone to Lose,Wilco,0.808,0.572,0,-7.969,1,0.377000,0.29800,0.0927,0.730,122.209,200355,4


#### Test Random Forest

In order to test my best classifier (Random Forest):

In [19]:
#using the best algorithm (have to train again...)

data = pd.read_csv('my_data.csv')

features = ['danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms', 'time_signature']

X = data[features]
y = data['Like']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

metrics.accuracy_score(y_test, y_pred)

0.8830188679245283

In [18]:
#passing with the test playlist 

pred = clf.predict(test_df[features])

test_df['prediction'] = pred
test_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.6250,0.03550,0.116,0.502,143.802,216004,4,1
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.0443,0.00826,0.142,0.369,125.389,177520,4,1
0,6hlJIYn5IBNqcSGTS6X5Xq,My Girls,Animal Collective,0.508,0.953,4,-6.031,0,0.2560,0.04450,0.929,0.555,119.973,340787,4,0
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.0577,0.01970,0.343,0.537,117.783,315200,4,1
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.1110,0.04490,0.105,0.301,139.389,335333,4,1


In [22]:
liked_test_songs = test_df[test_df['prediction'] ==1]
liked_test_songs

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.6250,0.03550,0.1160,0.502,143.802,216004,4,1
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.0443,0.00826,0.1420,0.369,125.389,177520,4,1
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.0577,0.01970,0.3430,0.537,117.783,315200,4,1
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.1110,0.04490,0.1050,0.301,139.389,335333,4,1
0,0gBrJS89BL4r8rqjDEzbtp,Put Your Number in My Phone,Ariel Pink,0.419,0.866,10,-3.189,0,0.0493,0.34400,0.1400,0.838,149.897,172827,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,6YQeOwMMAkB9MV9yMWmrjh,Threat of Joy,The Strokes,0.594,0.728,0,-6.917,1,0.2160,0.08810,0.1240,0.687,137.970,264662,4,1
0,1LfWibYjk0TD8YNPtzym9A,33 “GOD”,Bon Iver,0.520,0.389,4,-10.476,1,0.8300,0.02650,0.0937,0.326,133.113,213000,4,1
0,5HbKn6gfDgWJ8jJ8VUflPt,Gaia Cosmica,"Cachupa Psicadélica, João Gomes",0.618,0.741,11,-8.568,0,0.2810,0.15800,0.1770,0.372,116.023,244000,4,1
0,7zqhUgWpWzwYnT9eewPJEK,The End,Daughter,0.463,0.688,9,-8.127,1,0.8730,0.00456,0.0666,0.200,128.986,329647,4,1
